In [1]:
import numpy as np
import random 

In [2]:
a = np.array([[1, 0, -1, -28, 0],
              [0, 1, 2,   6, 0],
              [0, 0, 0, 9/2, 1]])
x1 = np.array([[-17/3], [3], [0], [1/3], [0]])
x2 = np.array([[1], [-2], [1], [0], [0]]) * random.random()
x3 = np.array([[-58/9], [4/3], [0], [-2/9], [1]]) * random.random()
x = x1+x2+x3
xb = np.array([[-15], [5], [0], [0], [1.5]])
a@x, x2, x3

(array([[-15.02040457],
        [  5.        ],
        [  1.5       ]]),
 array([[ 0.84515957],
        [-1.69031913],
        [ 0.84515957],
        [ 0.        ],
        [ 0.        ]]),
 array([[-0.59173247],
        [ 0.12242741],
        [ 0.        ],
        [-0.02040457],
        [ 0.09182055]]))

In [4]:
from gym_tictactoe.env import TicTacToeEnv, agent_by_mark, next_mark

env = TicTacToeEnv()
obs = env.reset()

In [5]:
env.action_space

Discrete(9)

In [6]:
class RolloutPolicy:
    def __init__(self, env):
        self.env = env
    def act(self, obs, render=False):
        return env.action_space.sample()
    def rollout(self, obs, model, render=False):
        d = False
        rsum = 0
        while not d:
            obs, r, d, _ = model.step(obs, self.act(obs))
            rsum += r
        return rsum

class Model:
    def __init__(self, env):
        self.env = env
    def step(self, obs, action):
        self._set_env(obs)
        return self.env.step(action)
    def available_actions(self, obs):
        self._set_env(obs)
        return self.env.available_actions()
    def _set_env(self, obs):
        self.env.board = list(obs[0])
        self.env.mark  = obs[1] 
        self.done = False
    def get_num_actions(self):
        return self.env.action_space.n

class TreePolicy:
    def __init__(self):
        pass
    def act(self, obs, available_actions):
        import random
        return random.choice(available_actions)
    def get_action_probs(self, obs, available_actions):
        return [1/len(available_actions) for i in range(len(available_actions))]

rollout_policy = RolloutPolicy(env)
model = Model(TicTacToeEnv())
tree_policy = TreePolicy()
# obs = env.reset()
# rollout_policy.rollout(obs, model)

In [29]:
class Node:
    def __init__(self, obs, reward, change_child_rew_sign=True, reward_sign=1, done=False, parent=None):
        self.n = 0
        self.cumulative_reward = 0#reward
        self.parent = parent
        self.action2child = {}
        self.nchildren = 0
        self.taken_actions = []
        self.obs = obs
        self.done = done
        self.reward = reward
        self.reward_sign = reward_sign
        self.change_child_rew_sign = change_child_rew_sign
    def get_q(self):
        return self.cumulative_reward/(self.n)

    def backpropagate(self, r, gamma=1):
        self.n += 1
        self.cumulative_reward += r*gamma*self.reward_sign
        if not( -1 <= self.get_q() <= 1 ) and 0:
            print(self.__dict__)
        if self.parent is None:
            return 
        self.parent.backpropagate(r, gamma=gamma)
    def print_parents(self):
        if self.parent is None:
            return
        self.parent.print_parents()

    def create_child(self, ChildType, policy, model):
        non_taken_actions = self._list_cut(model.available_actions(self.obs), self.taken_actions)
        action = policy.act(self.obs, non_taken_actions)
        self.taken_actions.append(action)
        obs, reward, done, info = model.step(self.obs, action)
        reward_sign= -self.reward_sign if self.change_child_rew_sign else self.reward_sign
        child = ChildType(obs, reward, done=done, reward_sign=reward_sign, parent=self)
        self.action2child[action] = child
        self.nchildren += 1
        return child, action

    def _list_cut(self, l1, l2):
        toret = []
        for a1 in l1:
            if a1 not in l2:
                toret.append(a1)
        return toret

    def rollout(self, rollout_policy, render=False):
        if self.done:
            return self.reward
        return rollout_policy.act(self.obs, render=render)

In [30]:
def UCB(root_node, policy, model, alpha=1):
    scores = [0 for i in range(model.get_num_actions())]
    all_actions = list(range(model.get_num_actions()))
    probs = policy.get_action_probs(root_node.obs, all_actions)
    maxscore, a_maxscore = float('-inf'), -1
    for a in root_node.action2child.keys():
        child = root_node.action2child[a]
        u = probs[a]/(1+child.n)
        q = child.get_q()
        score = q + alpha*u
        scores.append(score)
        if score > maxscore:
            maxscore = score
            a_maxscore = a
    return scores, (a_maxscore, maxscore)

def MCTS(root_node, max_depth, n_times, policy, model, alpha_UCB=1):
    current_node  = root_node
    current_depth = 0
    n_times_done  = 0

    while n_times_done != n_times:
        if current_depth == max_depth or current_node.done:
            current_node.print_parents()
            reward = current_node.rollout(rollout_policy)
            current_node.backpropagate(reward, gamma=0.99)
            current_node = root_node
            n_times_done += 1
            current_depth = 0
            model.env.done = False
        elif current_node.nchildren < len(model.available_actions(current_node.obs)):
            child, action = current_node.create_child(Node, policy, model) 
            current_node = child
            current_depth += 1
        else:
            scores, (a_maxscore, maxscore) = UCB(current_node, policy, model, alpha_UCB)
            current_node = current_node.action2child[a_maxscore]
            current_depth += 1

    visits = []
    for a in range(model.get_num_actions()):
        if a in root_node.action2child:
            visits.append(root_node.action2child[a].get_q())
        else:
            visits.append(float('nan'))
    return visits

In [33]:
import numpy as np
test_env = TicTacToeEnv()
obs = test_env.reset()
# obs, r, done, _ = test_env.step(4)
# obs, r, done, _ = test_env.step(0)
# obs, r, done, _ = test_env.step(1)
# obs, r, done, _ = test_env.step(7)
# obs, r, done, _ = test_env.step(6)
# obs, r, done, _ = test_env.step(2)
# obs, r, done, _ = test_env.step(3)
# obs, r, done, _ = test_env.step(5)
done = False
test_env.render()

while not done:
    root = Node(obs, 0)
    dic = MCTS(root, 10, 10000, tree_policy, model, 100)
    dic = np.array(dic)
    if test_env.mark != test_env.start_mark:
        dic *= -1

    dic = np.nan_to_num(dic, nan=100)
    move = np.argmin(dic)
    print(move, test_env.mark)
    print(dic.reshape(3, 3))
    obs, r, done, _ = test_env.step(move)
    test_env.render()
    # print(obs, done, r)
    print(" ")
    print(" ")
    print(" ")

   | | 
  -----
   | | 
  -----
   | | 

4 O
[[-0.3689441  -0.30518797 -0.37125   ]
 [-0.3025     -0.58384615 -0.30046433]
 [-0.35221154 -0.3006228  -0.32659794]]
   | | 
  -----
   |O| 
  -----
   | | 

 
 
 
0 X
[[  0.10182792   0.57315789   0.56163462]
 [  0.59921053 100.           0.57483871]
 [  0.48         0.87576923   0.476     ]]
  X| | 
  -----
   |O| 
  -----
   | | 

 
 
 
1 O
[[ 1.00000000e+02 -5.99210526e-01 -5.99210526e-01]
 [-5.62500000e-01  1.00000000e+02 -4.86885246e-01]
 [-5.79512195e-01 -4.66714286e-01 -2.78399258e-02]]
  X|O| 
  -----
   |O| 
  -----
   | | 

 
 
 
7 X
[[1.00000000e+02 1.00000000e+02 7.02580645e-01]
 [5.62941176e-01 1.00000000e+02 7.61538462e-01]
 [5.18571429e-01 1.25265306e-02 7.16896552e-01]]
  X|O| 
  -----
   |O| 
  -----
   |X| 

 
 
 
6 O
[[ 1.00000000e+02  1.00000000e+02 -4.40000000e-01]
 [-5.50000000e-01  1.00000000e+02 -4.80000000e-01]
 [-7.29473684e-01  1.00000000e+02 -6.61001517e-03]]
  X|O| 
  -----
   |O| 
  -----
  O|X| 

 
 
 
2 X
[[

In [490]:
env.render()
root = Node(obs, 0)
dic = MCTS(root, 100, 1000, tree_policy, model, 1)
dic = np.array(dic)
move = np.argmin(dic)
print(done, move)
print("")
obs, r, done, _ = env.step(move)
env.render()

  O|O|X
  -----
  X|O|X
  -----
  X| |X

True 0

  X|O|X
  -----
  X|O|X
  -----
  X| |X



In [622]:
print(dic)
dic[dic==float('nan')] = float('inf')
dic

[0.87096774 0.84375           nan 0.92307692 0.8852459  0.82442748
 0.75524476 0.93162393 0.91525424]


array([0.87096774, 0.84375   ,        nan, 0.92307692, 0.8852459 ,
       0.82442748, 0.75524476, 0.93162393, 0.91525424])

In [623]:
dic.nan

array([0.87096774, 0.84375   ,        nan, 0.92307692, 0.8852459 ,
       0.82442748, 0.75524476, 0.93162393, 0.91525424])

In [282]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [40]:
obs

(((0, 1, 0, 0, 0, 0, 0, 0, 0), 'X'), 0, False, None)

In [86]:
env.available_actions()

[3, 4, 5]

In [150]:
a = {1:12, 2:23}


In [156]:
for i in a:
    print(i)

1
2


In [333]:
obs = env.reset()
for i in range(9):
    print(env.step((i+1)%9))

(((0, 1, 0, 0, 0, 0, 0, 0, 0), 'X'), 0, False, None)
(((0, 1, 2, 0, 0, 0, 0, 0, 0), 'O'), 0, False, None)
(((0, 1, 2, 1, 0, 0, 0, 0, 0), 'X'), 0, False, None)
(((0, 1, 2, 1, 2, 0, 0, 0, 0), 'O'), 0, False, None)
(((0, 1, 2, 1, 2, 1, 0, 0, 0), 'X'), 0, False, None)
(((0, 1, 2, 1, 2, 1, 2, 0, 0), 'O'), -1, True, None)
(((0, 1, 2, 1, 2, 1, 2, 0, 0), 'O'), 0, True, None)
(((0, 1, 2, 1, 2, 1, 2, 0, 0), 'O'), 0, True, None)
(((0, 1, 2, 1, 2, 1, 2, 0, 0), 'O'), 0, True, None)
